In [1]:
import numpy as np
import pandas as pd
import random as rnd

#Rule for this notebook: If something is commented out, don't touch it unless you know 100% what it does - kai.

In [2]:
Local_dir = "K:\\Users\KJR\\Documents\\CSVs\\DATA3001\\" #replace however you need to
Event_list = pd.read_csv(Local_dir + "2019-2021_EventData.csv")
match_list = pd.read_csv(Local_dir + "2019-2021_MatchListing.csv")

In [3]:
miss_index = Event_list[Event_list['EventCode'] == 'TRY']
miss_index =  pd.concat([miss_index, Event_list[Event_list['EventCode'] == 'CVMS']])
miss_index = miss_index.sort_index(axis = 0)
misses = miss_index[['EventCode','PositionId']]

In [4]:
ind = list(misses.index)
removals = []
misses.index = range(len(misses))
for i in misses.index:
    if misses['EventCode'][i] == 'TRY' and i != len(misses)-1:
        if misses['EventCode'][i+1] != 'CVMS':
            removals.append(ind[i])
for i in removals:
    ind.remove(i)

In [5]:
removals = []
for i in range(len(ind)):
    if i % 2 == 1:
        removals.append(ind[i])
for i in removals:
    ind.remove(i)

In [6]:
ind.remove(3394249)
print(len(ind))
print(len(Event_list[Event_list['EventCode'] == 'CVMS']))

556
556


In [7]:
miss_index = Event_list[Event_list['EventCode'] == 'TRY']
miss_index =  pd.concat([miss_index, Event_list[Event_list['EventCode'] == 'CVMS']])
miss_index = miss_index.sort_index(axis = 0)
misses = miss_index[['EventCode','PositionId']]
misses.index

Int64Index([   6635,    7961,    9204,   11026,   11036,   12684,   14032,
              14043,   17155,   17164,
            ...
            3383406, 3385554, 3385563, 3386564, 3386575, 3388581, 3392810,
            3393723, 3393732, 3394249],
           dtype='int64', length=2723)

In [8]:
misses['Inde'] = list(misses.index)
misses

<ipython-input-8-eff12d7efb9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misses['Inde'] = list(misses.index)


,EventCode,PositionId,Inde
6635,TRY,61.0,6635
7961,TRY,64.0,7961
9204,TRY,61.0,9204
11026,TRY,54.0,11026
11036,CVMS,59.0,11036
...,...,...,...
3388581,TRY,54.0,3388581
3392810,TRY,53.0,3392810
3393723,TRY,53.0,3393723
3393732,CVMS,59.0,3393732


In [9]:
misses = misses.loc[misses.Inde.isin(ind)]

In [10]:
misses

,EventCode,PositionId,Inde
11026,TRY,54.0,11026
14032,TRY,55.0,14032
17155,TRY,54.0,17155
17944,TRY,55.0,17944
19974,TRY,61.0,19974
...,...,...,...
3379490,TRY,53.0,3379490
3383395,TRY,59.0,3383395
3385554,TRY,56.0,3385554
3386564,TRY,56.0,3386564


In [11]:
Position_list = Event_list[Event_list['Points'] > 0]
Position_list.to_csv('Point_Event_list.csv')
#Position_list = Position_list[Position_list['SeasonId'] == 2019]
Scoring_indicies = Position_list.index
Scoring_indicies

Int64Index([   3850,    6226,    6635,    6652,    7961,    7972,    9204,
               9213,   11026,   12684,
            ...
            3385142, 3385554, 3386564, 3388581, 3388590, 3392810, 3392821,
            3393723, 3394249, 3394260],
           dtype='int64', length=4183)

In [12]:
rnd.seed(3) #Please use same seed
samp_size = 4183
Play_sample = rnd.sample(list(Scoring_indicies), samp_size)
Play_sample.sort()
plays = Event_list[Event_list.index.isin(Play_sample)]

In [13]:
i = 0
prev_plays = 100
for x in Play_sample:
    if i == samp_size: #Small check as play_sample is infinitely increasing
        break;
    for y in np.arange(x-prev_plays, x, 1):
        Play_sample.append(y)
    i += 1
Play_sample.sort()
print(len(Play_sample))

422483


In [14]:
sample = Event_list[Event_list.index.isin(Play_sample)]
Tallies_all = pd.DataFrame(columns = sample['EventCode'].unique())

In [15]:
for x in np.arange(0,samp_size,1):
    small_sample = sample.iloc[x*(prev_plays+1)+1:x*(prev_plays+1)+prev_plays] #Grabs the plays based on the prev_plays variable, tallies them up to the next position scoring play
    count = small_sample['EventCode'].value_counts()
    Tallies = pd.DataFrame(count)
    Tallies_all = Tallies_all.append(Tallies.transpose())
Tallies_all.index = plays.index



In [16]:
NoOpponents = pd.read_csv('NoOppEvents.csv')

NoOppCodes = NoOpponents['Code'].tolist()

NoOppCodes

for col in Tallies_all.columns:
    if col not in NoOppCodes:
        Tallies_all = Tallies_all.drop(col, axis = 1)
Tallies_all.columns

Index(['PBNS', 'RCVS', 'RSPS', 'MACS', 'HTPS', 'MBCS', 'OADS', 'TKES', 'TKED',
       'REFT',
       ...
       'VCHS', 'CVMS', 'P3LS', 'RINS', 'RCAS', 'RKTS', 'SBKS', 'TTSD', 'I10S',
       'RSLS'],
      dtype='object', length=176)

In [17]:
Tallies_all = Tallies_all.fillna(0)
Tackles = ['TKES', 'REFT', 'TK1S', 'TKOS', 'FLPS', 'PFTA', 'TKSS', 'TKPS', 'TTSS', 'TKTS']
Passes = ['PSOS', 'PSFS', 'P1OS', 'P2OS', 'P1FS', 'P2FS', 'LAPS', 'PFWS', 'P3OS', 'P3FS', 'PSLS', 'P2LS', 'P3LS', 'P1LS', 'PSKS']
Dummy_passes = ['PDOS', 'DPLS', 'DPRS', 'PDFS', 'PDLS']
Ruck_Infringements = ['DOPS', 'RINS']

Tallies_all['Tackle_sum'] = 0

for col in Tackles:
    Tallies_all['Tackle_sum'] = Tallies_all['Tackle_sum'] + Tallies_all[col]

Tallies_all['Pass_sum'] = 0

for col in Passes:
    Tallies_all['Pass_sum'] = Tallies_all['Pass_sum'] + Tallies_all[col]
Tallies_all['Dummy_pass_sum'] = 0
for col in Dummy_passes:
    Tallies_all['Dummy_pass_sum'] = Tallies_all['Dummy_pass_sum'] + Tallies_all[col]

Tallies_all['RuckInfs'] = 0
for col in Ruck_Infringements:
    Tallies_all['RuckInfs'] = Tallies_all['RuckInfs'] + Tallies_all[col]
    
for col in Tallies_all.columns:
    if col  in Tackles:
        Tallies_all = Tallies_all.drop(col, axis = 1)
for col in Tallies_all.columns:
    if col  in Passes:
        Tallies_all = Tallies_all.drop(col, axis = 1)
for col in Tallies_all.columns:
    if col  in Ruck_Infringements:
        Tallies_all = Tallies_all.drop(col, axis = 1)

Tallies_all.columns

Index(['PBNS', 'RCVS', 'RSPS', 'MACS', 'HTPS', 'MBCS', 'OADS', 'TKED', 'MKRS',
       'MKRD',
       ...
       'RCAS', 'RKTS', 'SBKS', 'TTSD', 'I10S', 'RSLS', 'Tackle_sum',
       'Pass_sum', 'Dummy_pass_sum', 'RuckInfs'],
      dtype='object', length=153)

In [18]:
tallies = Tallies_all.columns
tallies.drop(['Pass_sum', 'Dummy_pass_sum', 'Tackle_sum', 'RuckInfs', 'I10S', 'CVMS'])
linebreaks = ['LAPS', 'LBKS', 'LAOS', 'LBIS', 'KK4S']

for col in Tallies_all.columns:
    if col not in linebreaks:
        if (np.sum(Tallies_all[col]) < int(4183/2)):
            Tallies_all.drop(col,inplace=True,axis=1)

In [19]:
contact_events = ['MBCS', 'MACS', 'MKRD', 'TKMD', 'ENCY', 'STCY']

for col in Tallies_all.columns:
    if col in contact_events:
        Tallies_all = Tallies_all.drop(col, axis = 1)

In [20]:
sample = pd.concat([plays,Tallies_all], axis=1)
Unnecessary_Events = ['VenueId','Team A Name', 'Team B Name', 'SeriesId', 'WeatherConditionName',
                      'SeriesName', 'Jumper', 'VenueId', 'Qualifier1Name', 'Qualifier1', 'Qualifier2Name', 'Qualifier2',
                      'Qualifier3Name', 'Qualifier3', 'Qualifier4Name', 'Qualifier4', 'Qualifier5Name', 'Qualifier5',
                      'Qualifier6Name', 'Qualifier6', 'Qualifier7Name', 'Qualifier7',
                      'Qualifier8Name', 'Qualifier8', 'DurationSecs', 'DistanceMs', 'OfficialId', 'Team A id', 'Team B ID',
                      'Club Name', 'Opposition Id', 'InPossessionClub Id', 'InPossession Player Id', 'SeqNumber'
                     , 'ElapsedSecs', 'GameSecs', 'ChannelPhysical', 'SectionPhysical', 'ChannelPlayer', 'ZonePhysical', 'XmPlayer'
                     , 'Player Id', 'RunOn', 'EventName', 'YmPhysical', 'Club Id', 'TeamAScore', 'TeamBScore', 'MatchId', 'XmPhysical', 'SectionPhysical', 'XmPossession',
                     'SectionPlayer', 'YmPossession', 'ZonePossession', 'SectionPossession', 'YmPlayer', 'ZonePlayer',  'ElapsedMins', 'GameMins', 'Set', 'Score', 'OppScore'
                     , 'OppPossessionSecs', '0_20_ElapsedSecs', '20_Half_ElapsedSecs','Half_20_ElapsedSecs', '20_Try_ElapsedSecs'] #So far, will trim as we get further along project
sample = sample.drop(Unnecessary_Events, axis=1)
sample
Tallies_all

,PBNS,RCVS,RSPS,HTPS,OADS,TKED,MKRS,ENPO,STPO,RUNS,...,TKMS,BKCS,LBKS,LAOS,KK4S,PDLS,LBIS,Tackle_sum,Pass_sum,Dummy_pass_sum
3850,3,7,4,3,3,3,6,1,1,0,...,0,0,0,0,0,0,0,11,0,0
6226,1,7,2,2,4,2,4,1,1,0,...,0,0,0,0,0,0,0,7,0,0
6635,3,7,5,2,3,3,6,0,0,1,...,0,0,0,0,0,0,0,11,0,0
6652,2,6,3,2,1,2,4,2,3,0,...,0,0,0,0,0,0,0,6,3,0
7961,3,6,5,2,2,3,5,1,1,0,...,1,1,1,0,0,0,0,8,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3392810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3392821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3393723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3394249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
ScoreCodes = pd.get_dummies(sample['EventCode'])
HalfCodes = pd.get_dummies(sample['Half'])
PositionCodes = pd.get_dummies(sample['ChannelPossession'])
WeatherCodes = pd.get_dummies(sample['WeatherConditionId'])

ScoreCodes.columns = 'Goal_' + ScoreCodes.columns
for col in HalfCodes.columns:
    col1 = 'Half_' + str(col)
    HalfCodes = HalfCodes.rename({col:col1}, axis = 1)
for col in WeatherCodes.columns:
    col1 = 'Weather_' + str(col)
    WeatherCodes = WeatherCodes.rename({col:col1}, axis = 1)
sample = pd.concat([sample, PositionCodes, ScoreCodes, WeatherCodes, HalfCodes], axis = 1)
sample = sample.drop(['EventCode', 'ChannelPossession', 'WeatherConditionId', 'Half'], axis = 1)

In [22]:
sample['Missed_convert'] = 0
for i in misses.Inde:
    sample['Missed_convert'][i] = 1

sample

<ipython-input-22-10c0abf4859e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['Missed_convert'][i] = 1


,SeasonId,RoundId,PositionId,Captain,ElapsedMillisecs,Tackle,Points,PossessionSecs,TotalPossessionSecs,PBNS,...,Goal_TRY,Weather_2,Weather_3,Weather_4,Weather_1040,Half_1,Half_2,Half_3,Half_4,Missed_convert
3850,2019,1,59.0,Y,1771200,0,2.0,537.82,1142.02,3,...,0,0,0,1,0,1,0,0,0,0
6226,2019,1,56.0,N,290720,0,2.0,991.28,1855.64,1,...,0,0,0,1,0,0,1,0,0,0
6635,2019,1,61.0,N,486880,6,4.0,1068.54,1982.58,3,...,1,0,0,1,0,0,1,0,0,0
6652,2019,1,56.0,N,582440,0,2.0,1068.62,1982.66,2,...,0,0,0,1,0,0,1,0,0,0
7961,2019,1,64.0,N,1379560,3,4.0,1313.64,2368.24,3,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3392810,2021,25,53.0,N,2030960,0,4.0,1444.38,3115.62,0,...,1,1,0,0,0,0,1,0,0,0
3392821,2021,25,59.0,N,2088680,0,2.0,1444.46,3115.70,0,...,0,1,0,0,0,0,1,0,0,0
3393723,2021,25,53.0,N,2431400,3,4.0,1600.90,3389.56,0,...,1,1,0,0,0,0,1,0,0,1
3394249,2021,25,57.0,N,2791920,5,4.0,1709.26,3557.92,0,...,1,1,0,0,0,0,1,0,0,0


In [23]:
sample.index = range(len(sample))

In [24]:
sample['TRY_Convert'] = 0
length = len(sample)

for i in range(length):
        if sample['Goal_TRY'][i] == 1:
            sample['TRY_Convert'][i] = sample['Goal_CVOK'][i+1]
            
small = sample[['Goal_TRY', 'Goal_CVOK', 'TRY_Convert', 'Missed_convert']]
#small = small[small['Goal_TRY'] == 1]

<ipython-input-24-f0bafc7c4c51>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['TRY_Convert'][i] = sample['Goal_CVOK'][i+1]


In [25]:
sample.loc[sample['Captain'] == 'Y', 'Captain'] = 1
sample.loc[sample['Captain'] == 'N', 'Captain'] = 0
sample = sample.drop(sample[sample['PositionId'] == 66].index)
sample.to_csv('sample_all_Positions.csv')

In [26]:
back = [53,54,55,56,57]
mid = [58,59,65]
front = [60,61,62,63,64]

for i in back:
    sample.loc[sample['PositionId'] == i, 'PositionId'] = 2

for j in mid:
    sample.loc[sample['PositionId'] == j, 'PositionId'] = 1
    
for k in front:
    sample.loc[sample['PositionId'] == k, 'PositionId'] = 0



sample = sample.fillna(0)
sample.to_csv('sample_all.csv')

In [27]:
sample = sample[sample['Goal_CVOK'] == 0]
sample

,SeasonId,RoundId,PositionId,Captain,ElapsedMillisecs,Tackle,Points,PossessionSecs,TotalPossessionSecs,PBNS,...,Weather_2,Weather_3,Weather_4,Weather_1040,Half_1,Half_2,Half_3,Half_4,Missed_convert,TRY_Convert
0,2019,1,1.0,1,1771200,0,2.0,537.82,1142.02,3,...,0,0,1,0,1,0,0,0,0,0
1,2019,1,2.0,0,290720,0,2.0,991.28,1855.64,1,...,0,0,1,0,0,1,0,0,0,0
2,2019,1,0.0,0,486880,6,4.0,1068.54,1982.58,3,...,0,0,1,0,0,1,0,0,0,1
4,2019,1,0.0,0,1379560,3,4.0,1313.64,2368.24,3,...,0,0,1,0,0,1,0,0,0,1
6,2019,1,0.0,0,1919840,4,4.0,1554.06,2752.46,1,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2021,25,2.0,0,1881720,0,4.0,681.22,1271.84,0,...,1,0,0,0,1,0,0,0,1,0
4176,2021,25,2.0,0,133880,6,4.0,967.40,1863.38,0,...,1,0,0,0,0,1,0,0,0,1
4178,2021,25,2.0,0,2030960,0,4.0,1444.38,3115.62,0,...,1,0,0,0,0,1,0,0,0,1
4180,2021,25,2.0,0,2431400,3,4.0,1600.90,3389.56,0,...,1,0,0,0,0,1,0,0,1,0


In [28]:
sample.to_csv('sample_all_NoConv.csv')

In [29]:
small = sample[sample['Goal_TRY'] == 1]
small = small[small['TRY_Convert'] == 0]
small = small[small['Missed_convert'] == 0]

In [30]:
small

,SeasonId,RoundId,PositionId,Captain,ElapsedMillisecs,Tackle,Points,PossessionSecs,TotalPossessionSecs,PBNS,...,Weather_2,Weather_3,Weather_4,Weather_1040,Half_1,Half_2,Half_3,Half_4,Missed_convert,TRY_Convert
1951,2020,10,2.0,0,2955320,3,4.0,2031.86,3666.10,1,...,1,0,0,0,0,1,0,0,0,0
2674,2020,24,1.0,0,3323560,1,4.0,1653.86,3306.40,2,...,0,1,0,0,0,1,0,0,0,0
2987,2021,6,0.0,0,61720,5,4.0,1744.42,3522.74,0,...,1,0,0,0,0,0,1,0,0,0
